In [30]:
import random
import numpy as np
from collections import defaultdict

# Methods

In [41]:
# converts a bipartite network into a transactional database
def from_bip_to_db(f, idx):
    transactions = defaultdict(set)
    with open(f) as in_f:
        for line in in_f.readlines():
            if "%" in line: continue
            line = line.strip()
            lst = line.split()
            item = int(lst[(idx + 1) % 2])
            trans = int(lst[idx])
            transactions[trans].add(item)
    return np.array(list(transactions.values()))

In [98]:
# retain leng random transactions in the database
def subsample(transactions, leng):
    if len(transactions) > leng:
        sample_idx = set()
        while len(sample_idx) < leng:
            sample_idx.add(random.randint(0, len(transactions)-1))
        return transactions[list(sample_idx)]
    return transactions

In [81]:
# given a transactional database, finds left and right degree distribution, and BJDM 
def compute_degrees_and_BJDM(transactions):
    edges = list()
    left_degrees = defaultdict(int)
    right_degrees = defaultdict(int)
    for idx, t in enumerate(transactions):
        left_degrees[idx] = len(t)
        for item in t:
            edges.append((idx, item))
            right_degrees[item] += 1
    max_len = max(left_degrees.values())
    max_deg = max(right_degrees.values())
    BJDM = np.zeros((max_len, max_deg))
    for edge in edges:
        l = left_degrees[edge[0]]-1
        r = right_degrees[edge[1]]-1
        BJDM[l][r] += 1
    ld = np.zeros(max_len)
    rd = np.zeros(max_deg)
    for v in left_degrees.values():
        ld[v-1] += 1
    for v in right_degrees.values():
        rd[v-1] += 1
    ld = ld.reshape(-1,1)
    rd = rd.reshape(-1,1)
    return BJDM, ld, rd

In [94]:
def write_db(f, transactions):
    with open(f, 'w') as out_f:
        for t in transactions:
            out_f.write(' '.join([str(x) for x in t])+'\n')

# Setup

In [23]:
data_dir = '../datasets/bipartite'

In [24]:
bip_graphs = ['gottron-reuters', 'bag-kos', 'dbpedia-occupation', 
              'movielens-100k_rating', 'edit-iewikibooks', 
              'librec-filmtrust-ratings', 'wang-tripadvisor', 
              'moreno_crime']
# if the transaction is the left or right node
side = {
    'gottron-reuters' : 0,
    'bag-kos' : 0,
    'dbpedia-occupation' : 0,
    'movielens-100k_rating' : 0,
    'edit-iewikibooks' : 0,
    'librec-filmtrust-ratings' : 0,
    'wang-tripadvisor' : 0,
    'moreno_crime' : 0
}

In [97]:
max_size = 4000

# Generation

In [99]:
dbs = dict()

In [101]:
for ds in bip_graphs:
    file_name = f'{data_dir}/{ds}/out.{ds}'
    # generate database
    trans = from_bip_to_db(file_name, side[ds])
    # sub-sample from db
    trans = subsample(trans, max_size)
    dbs[ds] = trans
    # compute bjdm and marginals - normalize!
    bjdm, ld, rd = compute_degrees_and_BJDM(trans)
    bjdm_n = bjdm / np.sum(bjdm)
    ld_n = ld / np.sum(ld)
    rd_n = rd / np.sum(rd)
    # joint deg dist is the product of the marginals
    joint = (ld_n @ rd_n.T)
    print(bjdm_n.shape, round(np.sum(bjdm_n)), 
          joint.shape, round(np.sum(joint)))
    diff = bjdm_n - joint
    # the larger the distance between bjdm and joint, 
    # the higher the probability that ALICE and GMMT
    # give different results in significant itemset mining
    dist = round(np.linalg.norm(diff, ord='fro'), 5)
    # tentative threshold
    abs_diff = np.abs(diff) * np.sum(bjdm)
    thresh = np.mean(abs_diff) / len(trans)
    print(ds, len(trans), dist, thresh)

(295, 3850) 1 (295, 3850) 1
gottron-reuters 4000 0.05815 6.625028265615344e-05
(457, 2123) 1 (457, 2123) 1
bag-kos 3430 0.00958 0.00011779562486871651
(18, 548) 1 (18, 548) 1
dbpedia-occupation 4000 0.30429 0.00019782186067706216
(737, 583) 1 (737, 583) 1
movielens-100k_rating 943 0.01303 0.000333226717016403
(181, 7) 1 (181, 7) 1
edit-iewikibooks 137 0.49284 0.004783716913068563
(244, 1044) 1 (244, 1044) 1
librec-filmtrust-ratings 1508 0.09904 0.00016361178647119096
(13, 66) 1 (13, 66) 1
wang-tripadvisor 4000 0.25582 0.001108152571805589
(25, 18) 1 (25, 18) 1
moreno_crime 829 0.28021 0.0036225142425240654


# Save Candidate DB

In [102]:
for db in ['edit-iewikibooks', 'dbpedia-occupation', 
           'moreno_crime', 'wang-tripadvisor']:
    write_db(f'{data_dir}/{db}.txt', dbs[db])